In [3]:
import base64
from anthropic import Anthropic
from IPython.display import Image

client = Anthropic()
MODEL_NAME = "claude-3-opus-20240229"

In [30]:
user_query = str(input('Enter your question or query: '))

In [31]:
def retrieve_values_for_tools(query):
    return f"""You are given a sentence that contains a property, and you should generate the values in the form of a list of 3 strings, where 

    - the first string is the property name, 
    - the second string is the type of the property (str, int) and 
    - the third string is a short 1-line description of what the property is.

    Here are a couple of examples that illustrate this:

    <example>
    Sentence: "I enjoy reading sci-fi genre books".
    The output: ['book_genre_interest','str','The genre of books the user enjoys.']
    </example>

    <example>
    Sentence: "My goal today is to ensure that I can get my bicep-workout done.
    The output: ['today_goal','str',"The user's goal for today"]
    </example>

    Now perform the same action here:

    Sentence: {query}
    Output:
    """


In [33]:
retrieve_values_for_tools(user_query)

'You are given a sentence that contains a property, and you should generate the values in the form of a list of 3 strings, where \n\n    - the first string is the property name, \n    - the second string is the type of the property (str, int) and \n    - the third string is a short 1-line description of what the property is.\n\n    Here are a couple of examples that illustrate this:\n\n    <example>\n    Sentence: "I enjoy reading sci-fi genre books".\n    The output: [\'book_genre_interest\',\'str\',\'The genre of books the user enjoys.\']\n    </example>\n\n    <example>\n    Sentence: "My goal today is to ensure that I can get my bicep-workout done.\n    The output: [\'today_goal\',\'str\',"The user\'s goal for today"]\n    </example>\n\n    Now perform the same action here:\n\n    Sentence: I am really looking forward to watching Oppenheimer because I enjoy Nolan\'s movies.\n    Output:\n    '

Retrieving the list of values in order to feed into the 

In [34]:
response = client.messages.create(
    model=MODEL_NAME,
    max_tokens=2048,
    messages=[{'role':'user','content':retrieve_values_for_tools(user_query)}],
)
print(response.content[0].text)

['anticipated_movie', 'str', "The movie the user is looking forward to watching."]


In [ ]:
def claude_response()